# DECISION TREE MODEL

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from skopt import BayesSearchCV

In [2]:
bu_train = pd.read_csv(r"C:\Users\ADMIN\Downloads\Project1-HOMECREDIT\home-credit-default-risk\bureau_fe_train.csv")
app_train = pd.read_csv(r"C:\Users\ADMIN\Downloads\Project1-HOMECREDIT\home-credit-default-risk\application_fe_train.csv")
bu_test = pd.read_csv(r"C:\Users\ADMIN\Downloads\Project1-HOMECREDIT\home-credit-default-risk\bureau_fe_test.csv")
app_test = pd.read_csv(r"C:\Users\ADMIN\Downloads\Project1-HOMECREDIT\home-credit-default-risk\application_fe_test.csv")

In [3]:
other = pd.read_csv(r"C:\Users\ADMIN\Downloads\Project1-HOMECREDIT\home-credit-default-risk\feature_prev_cc_pos_install.csv")

In [28]:
merge_train = pd.merge(bu_train, app_train, how = 'left', on = 'SK_ID_CURR')

In [29]:
merge_train = pd.merge(merge_train, other, how = 'left', on = 'SK_ID_CURR')

In [12]:
merge_train.head()

,Unnamed: 0_x,SK_ID_CURR,TARGET_x,BURO_SK_ID_BUREAU_MEAN,BURO_SK_ID_BUREAU_MAX,BURO_SK_ID_BUREAU_MIN,BURO_SK_ID_BUREAU_SUM,BURO_SK_ID_BUREAU_MEDIAN,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_MAX,...,LAST_LOAN_DPD_std,LAST_LOAN_LATE_PAYMENT_count,LAST_LOAN_LATE_PAYMENT_mean,LAST_LOAN_PAID_OVER_AMOUNT_sum,LAST_LOAN_PAID_OVER_AMOUNT_mean,LAST_LOAN_PAID_OVER_AMOUNT_max,LAST_LOAN_PAID_OVER_AMOUNT_min,LAST_LOAN_PAID_OVER_AMOUNT_std,LAST_LOAN_PAID_OVER_count,LAST_LOAN_PAID_OVER_mean
0,0,100002,1,6153272.125,6158909.0,6113835.0,49226177.0,6158905.5,-874.00,-103.0,...,0.0,19.0,1.000000,0.0,0.0,0.0,0.0,0.0,19.0,0.0
1,1,100003,0,5885878.500,5885880.0,5885877.0,23543514.0,5885878.5,-1400.75,-606.0,...,0.0,7.0,1.000000,0.0,0.0,0.0,0.0,0.0,7.0,0.0
2,2,100004,0,6829133.500,6829134.0,6829133.0,13658267.0,6829133.5,-867.00,-408.0,...,0.0,3.0,1.000000,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3,3,100006,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,10.0,1.000000,0.0,0.0,0.0,0.0,0.0,10.0,0.0
4,4,100007,0,5987200.000,5987200.0,5987200.0,5987200.0,5987200.0,-1149.00,-1149.0,...,0.0,12.0,0.833333,0.0,0.0,0.0,0.0,0.0,12.0,0.0


In [30]:
merge_train = merge_train.drop(columns=["Unnamed: 0_x"])

In [31]:
merge_train.shape

(307511, 750)

In [32]:
merge_test = pd.merge(bu_test, app_test, how = 'left', on = 'SK_ID_CURR')

In [33]:
merge_test = pd.merge(merge_test, other, how = 'left', on = 'SK_ID_CURR')

In [40]:
merge_test.head()

,SK_ID_CURR,BURO_SK_ID_BUREAU_MEAN,BURO_SK_ID_BUREAU_MAX,BURO_SK_ID_BUREAU_MIN,BURO_SK_ID_BUREAU_SUM,BURO_SK_ID_BUREAU_MEDIAN,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_SUM,...,LAST_LOAN_DPD_std,LAST_LOAN_LATE_PAYMENT_count,LAST_LOAN_LATE_PAYMENT_mean,LAST_LOAN_PAID_OVER_AMOUNT_sum,LAST_LOAN_PAID_OVER_AMOUNT_mean,LAST_LOAN_PAID_OVER_AMOUNT_max,LAST_LOAN_PAID_OVER_AMOUNT_min,LAST_LOAN_PAID_OVER_AMOUNT_std,LAST_LOAN_PAID_OVER_count,LAST_LOAN_PAID_OVER_mean
0,100001,5896633.00,5896636.0,5896630.0,41276431.0,5896633.0,-735.000000,-49.0,-1572.0,-5145.0,...,0.000000,4.0,1.000000,0.00,0.000000,0.0,0.0,0.000000,4.0,0.0
1,100005,6735201.00,6735202.0,6735200.0,20205603.0,6735201.0,-190.666667,-62.0,-373.0,-572.0,...,0.333333,9.0,0.888889,0.00,0.000000,0.0,0.0,0.000000,9.0,0.0
2,100013,5922080.75,5922083.0,5922077.0,23688323.0,5922081.5,-1737.500000,-1210.0,-2070.0,-6950.0,...,0.093659,114.0,0.192982,-7875.00,-69.078947,0.0,-6750.0,639.995356,114.0,0.0
3,100028,6356884.50,6356890.0,6356879.0,76282614.0,6356884.5,-1401.750000,-269.0,-2105.0,-16821.0,...,1.168284,84.0,0.309524,-70348.23,-837.478929,0.0,-8505.0,1956.753696,84.0,0.0
4,100038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,12.0,1.000000,0.00,0.000000,0.0,0.0,0.000000,12.0,0.0


In [39]:
merge_test = merge_test.drop(columns=["Unnamed: 0_x"])

In [41]:
merge_test.shape

(48744, 748)

In [44]:
set(merge_train.columns).symmetric_difference(set(merge_test.columns))

{'TARGET_x', 'TARGET_y'}

In [47]:
merge_train = merge_train.drop(columns=["TARGET_y"])

In [48]:
merge_train = merge_train.rename(columns={"TARGET_x": "TARGET"})

In [55]:
merge_train = merge_train.replace([np.inf, -np.inf], np.nan)
merge_test = merge_test.replace([np.inf, -np.inf], np.nan)

In [56]:
id_col = "SK_ID_CURR"
y = merge_train["TARGET"]

feature_cols = [c for c in merge_train.columns if c not in ["TARGET"]]
X = merge_train[feature_cols]

X_test_full = merge_test[feature_cols]


## BAGGING FOR FEATURE SELECTION USING RANDOM FOREST

In [58]:
TOP_N = 300

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
    class_weight="balanced"
)

rf.fit(X, y)

importances = rf.feature_importances_
feat_importances = pd.Series(importances, index=feature_cols).sort_values(ascending=False)

selected_features = feat_importances.head(TOP_N).index.tolist()

print("Initial feature:", len(feature_cols))
print("Number of retain feature:", len(selected_features))
print("Top 20 most importance feature:")
print(feat_importances.head(20))

Initial feature: 748
Number of retain feature: 300
Top 20 most importance feature:
EXT_MEAN                        0.020884
EXT_PROD                        0.019467
EXT_WEIGHTED                    0.017841
EXT_MIN                         0.015167
EXT_MAX                         0.012328
EXT_SOURCE_2                    0.011179
EXT_SOURCE_2_MM                 0.010250
EXT_SOURCE_3_MM                 0.009734
EXT_SOURCE_3                    0.008323
BURO_DEBT_TO_CREDIT_MAX         0.005204
EXT_MEAN_MEDIAN                 0.004956
DAYS_EMPLOYED_YEARS             0.004935
BURO_DEBT_TO_CREDIT_MEAN        0.004679
CREDIT_TO_ANNUITY_RATIO_MEAN    0.004623
DAYS_EMPLOYED                   0.004623
EXT_STD                         0.004551
BURO_DAYS_CREDIT_MEDIAN         0.004499
DAYS_BIRTH                      0.004455
EXT_SOURCE_1                    0.004399
EXT_SOURCE_1_MM                 0.004237
dtype: float64


In [ ]:
#Retain only importance feature
X_sel = X[selected_features]
X_test_sel = X_test_full[selected_features]

In [64]:
opt = {
    "max_depth": (3, 20),
    "min_samples_split": (2, 100),
    "min_samples_leaf": (1, 50),
}

dt = DecisionTreeClassifier(class_weight="balanced", random_state=42)

bayes = BayesSearchCV(
    estimator=dt,
    search_spaces=opt,
    scoring="roc_auc",
    cv=5,
    n_iter=40,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

bayes.fit(X_sel, y)

print("Best params:", bayes.best_params_)
print("Best CV AUC:", bayes.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

In [65]:
best_model = bayes.best_estimator_
best_model.fit(X_sel, y)

,criterion,'gini'
,splitter,'best'
,max_depth,6
,min_samples_split,100
,min_samples_leaf,50
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,'balanced'


In [66]:
test_pred_proba = best_model.predict_proba(X_test_sel)[:, 1]

In [67]:
submission = pd.DataFrame({
    "SK_ID_CURR": merge_test["SK_ID_CURR"],
    "TARGET": test_pred_proba
})

In [68]:
submission.to_csv("submission_decision_tree_tuned.csv", index=False)